# Optimization with Linear Programming

## Case Study 1: Inventory and Transportation Problem
## Case Study 2: Portfolio Investment Problem
> ### Author: Lu (Christina) Jin

# Setup: Import all libraries

In [14]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pulp

In [24]:
import os
import io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy as scipy
import re
import sklearn as sk
import warnings

import datetime as dt
from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from datetime import date
from datetime import datetime
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


import statsmodels.api as sm
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib.ticker import PercentFormatter
from scipy import stats
from scipy.stats import bartlett
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK
from pulp import *

# 1. Transportation Problem

### Brief overview of the problem: 

It is a dataset with cost, capacity and demand information of each of the two types of plants (Marietta and Minneapolis) at each of the 4 distribution center (Cleveland, Baltimore, Chicago, and Phoenix). The objective is to minimize the cost. 

### Define the model:

**Objective (to minimize cost):**
> 12.60X1 + 14.35X2 + 11.52X3 + 17.58X4 + 9.75Y1 + 16.26Y2 + 8.11Y3 + 17.92Y4

**Supply Constraints:**
> X1 + X2 + X3 + X4 ≤ 1200
>
> Y1 + Y2 + Y3 + Y4 ≤ 800

**Demand Constraints:**
> X1 + Y1 = 150
>
> X2 + Y2 = 350
> 
> X3 + Y3 = 500
> 
> X4 + Y4 = 1000

**Positive Units Constraint:**
> X1, X2, X3, X4, Y1, Y2, Y3, Y4 ≥ 0


In [33]:
# Define the model
model = LpProblem(name="Transportation Problem", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {j: LpVariable(name=f"y{j}", lowBound=0) for j in range(1, 5)}

# Add constraints
model += (lpSum(x.values()) <= 1200, "Plant_Marietta")
model += (lpSum(y.values()) <= 800, "Plant_Minneapolis")
model += (x[1] + y[1] == 150, "Celeland_Distribution_Center")
model += (x[2] + y[2] == 350, "Baltimore_Distribution_Center")
model += (x[3] + y[3] == 500, "Chicago_Distribution_Center")
model += (x[4] + y[4] == 1000, "Phoenix_Distribution_Center")


# Set the objective
model += 12.6 * x[1] + 14.35 * x[2] + 11.52 * x[3] + 17.58 * x[4] + 9.75 * y[1] + 16.26 * y[2] +  8.11 * y[3] + 17.92 * y[4]

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")
    
for var in y.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 28171.0
x1: 0.0
x2: 350.0
x3: 0.0
x4: 850.0
y1: 150.0
y2: 0.0
y3: 500.0
y4: 150.0
Plant_Marietta: 0.0
Plant_Minneapolis: 0.0
Celeland_Distribution_Center: 0.0
Baltimore_Distribution_Center: 0.0
Chicago_Distribution_Center: 0.0
Phoenix_Distribution_Center: 0.0


### Solution:

> According to the result of our linear optimization model, minimized cost is going to be **$28171**, and the allocation of both plants are:
>
> **Plant Marietta:**
> * 350 units to be distributed at the Baltimore center
>
> * 850 units to be distributed at the Phoenix center
>
> **Plant Minneapolis:**
> * 150 units to be distributed at the Cleveland center
> 
> * 500 units to be distributed at the Chicago center
>
> * 150 units to be distributed at the Phoenix center
>
> 0 values in all slack variables meaning there would be no waste of any sort of resources.

# 2. Portfolio Investment Problem

### Brief overview of the problem: 

It is a dataset with expected annual return and risk measures of each of the 6 funds available. Our goal is to minimize the risk of investing $500,000 in these funds. While at the same time, to gain at least a 5% of average return, while meeting all the constrains on the portfolio allocation.

### Define the model:

**Objective (to minimize risk):**
> (10.57X1 + 13.22X2 + 14.02X3 + 2.39X4 + 9.30X5 + 7.61X6) / 500000

**Portfolio Allocation Constraints:**
> X1 + X2 + X3 + X4 + X5 + X6 = 500,000
>
> X2 ≥ 50,000
>
> X6 ≥ 50,000
>
> X5 + X6 ≥ 0.4 * 500,000

**Average Return Constraint:**
> (8.13X1 + 9.02X2 + 7.56X3 + 3.62X4 + 7.79X5 + 4.40X6)/500,000 ≥ 5.00

**Low and High Bound Constraints:**
> X1, X2, X3, X4, X5, X6 ≥ 0
>
> X1, X2, X3, X4, X5, X6 ≤ 200,000


In [51]:
# Define the model
model = LpProblem(name="Profolio Investment Problem", sense=LpMinimize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, upBound=200000) for i in range(1, 7)}

# Add constraints
model += (lpSum(x.values()) <= 500000, "Total_Investment")
model += (x[2] >= 50000, "Multinational_Fund")
model += (x[6] >= 50000, "Balanced_Fund")
model += (x[5] + x[6] >= 0.4*500000, "Income_Equity_&_Balanced_Funds")
model += (8.13 * x[1] + 9.02 * x[2] + 7.56 * x[3] + 3.62 * x[4] + 7.79 * x[5] + 4.40 * x[6] >= 5 * 500000, "Average Return")


# Set the objective
model += (10.57 * x[1] + 13.22 * x[2] + 14.02 * x[3] + 2.39 * x[4] + 9.30 * x[5] + 7.61 * x[6]) / 500000

# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 5.7451519168
x1: 0.0
x2: 50000.0
x3: 0.0
x4: 182458.56
x5: 150000.0
x6: 50000.0
Total_Investment: -67541.44
Multinational_Fund: 0.0
Balanced_Fund: 0.0
Income_Equity_&_Balanced_Funds: 0.0
Average_Return: -0.012800000113202259


### Solution:

> According to the result of our portfolio optimization model, the risk is going to be at **5.745** the lowest, and the average return will be 5.013% which would be beating the required percentage of return by 0.013%. 
>
> **The portfolio allocations are as follows:**
> * invest 50,000 dollars in Innis Multinational Fund
>
> * invest 182,458.56 dollars in Innis Mortgage Fund
>
> * invest 150,000 dollars in Innis Income Equity Fund
> 
> * invest 50,000 dollars in Innis Balance Fund
>
> **Other Things to Note:**
> * This plan of portfolio requires no money being invested for Innis Low-Priced Stock Fund or the Innis Mid-Cap Stock Fund (this is probably because these two funds are with pretty high risk measures.
>
> * There will be $67,541.44 left in the account without being invest in anything (this is probably because we are trying to minimize the risk)